In [1]:
import numpy as np
import dgl
import scanpy as sc
import pandas as pd

import torch 
import torch.nn as nn
import torch.nn.functional as F

Using backend: pytorch


In [2]:
adata = sc.read('/home/ngr4/project/sccovid/data/processed/hbec_scv2.h5ad')

In [20]:
# pre-processing
adata_test = adata[adata.obs['split']=='test', :]
sc.tl.pca(adata_test, n_comps=50)
sc.external.pp.bbknn(adata_test, n_pcs=50, neighbors_within_batch=int(30/len(adata_test.obs['batch'].unique())))

In [53]:
g = dgl.from_scipy(adata_test.obsp['connectivities'])
g.ndata['h'] = torch.tensor(adata_test.obsm['X_pca']) # test with just PCA-coords as node features 

In [54]:
# add label
label_encoder = {'mock':0, 
                 'bystander_1dpi':1, 
                 'infected_1dpi':2, 
                 'bystander_2dpi':3, 
                 'infected_2dpi':4, 
                 'bystander_3dpi':5, 
                 'infected_3dpi':6}
g.ndata['label'] = torch.LongTensor(adata_test.obs['inf'].map(label_encoder).tolist())

In [43]:
import sys
sys.path.append('/home/ngr4/project/dev/graphtransformer/scripts/')
from layers import GraphTransformerLayer

In [55]:
graphtransformer = GraphTransformerLayer(50, 50, 1)

In [59]:
graphtransformer(g, g.ndata['h'])

tensor([[-0.5042, -0.2999, -0.2316,  ..., -0.5872, -1.4215,  0.1535],
        [-0.2955, -1.9272, -0.3162,  ..., -0.0180,  0.5649, -0.5777],
        [-0.8107, -1.2720, -0.4066,  ...,  0.7604,  0.5405, -0.0444],
        ...,
        [ 1.2389, -1.0228, -1.1995,  ..., -0.1947,  0.4873,  0.5240],
        [ 1.3466, -0.0599, -0.2185,  ..., -0.4266, -0.6014, -0.6844],
        [ 0.7097,  0.4786, -0.0623,  ..., -1.7399, -0.2918, -0.6050]],
       grad_fn=<NativeBatchNormBackward>)